## Assignment

In [62]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install urllib
!pip install bs4

  ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matching distribution found for urllib


The first step is to import the libraries needed. We use beautiful soup to parse the HTML and scrape the website to extract all the URLs.

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen
from bs4 import BeautifulSoup
%matplotlib inline

We set the url to the website i.e wikipedia which contains all the links and information of cities of United States and access the site with our requests library.



In [2]:
#baseURL defined to handle later url calls for 2nd level scrapping
baseURL="https://en.wikipedia.org"

In [3]:
#main url for scrapping the initial table
url = "/wiki/List_of_United_States_cities_by_population"
html = urlopen(baseURL+url)

Inspect the site's HTML structure and parse it in a beautiful way.

In [4]:
soup = BeautifulSoup(html, 'lxml')

To find the table with all the top cities in the site and locate all the <tr\> tags (inspected the html to determine the name to ease the load on the runtime)

In [5]:
#fetches the table with the class name as wikitable which is the table the cities
rows = soup.find_all("table",class_="wikitable")

In [6]:
#fetches the first and only result out of the result set and fetches all the tr tags in the resultant table
trs = rows[1].find_all("tr")

Here we find all the links in each city, define the cells in the table and scrape the data.  We get all the table rows in list form and then convert into dataframe and for that we iterate through all the table rows.

In [7]:
city_list = []
for tr in trs:
    try:
        tds = tr.find_all("td")
        temp = {}
        temp["rank"] = tds[0].text.replace("\n","")
        temp["city"] = tds[1].text.replace("\n","")
        #code to find link to city
        temp["link"] = tds[1].find_all("a",href=True)[0]['href']
        temp["state"] = tds[2].text.replace("\n","").replace("\xa0","")
        temp["estimate"] = tds[3].text.replace("\n","")
        temp["census"] = tds[4].text.replace("\n","")
        temp["change"] = tds[5].text.replace("\n","")
        temp["land_area_mi"] = tds[6].text.replace("\n","").replace("\xa0","")
        temp["land_area_km2"] = tds[7].text.replace("\n","").replace("\xa0","")
        temp["population_density_mi"] = tds[8].text.replace("\n","").replace("\xa0","")
        temp["population_density_km2"] = tds[9].text.replace("\n","")
        temp["location"] = tds[10].text.replace("\ufeff","").replace("\n","")


        city_list.append(temp)
    except:
        continue

Generated a list of cities and its information.

In [8]:
city_list[0]

{'rank': '1',
 'city': 'New York City[d]',
 'link': '/wiki/New_York_City',
 'state': 'New York',
 'estimate': '8,398,748',
 'census': '8,175,133',
 'change': '+2.74%',
 'land_area_mi': '301.5sqmi',
 'land_area_km2': '780.9km2',
 'population_density_mi': '28,317/sqmi',
 'population_density_km2': '10,933/km2',
 'location': '40°39′49″N 73°56′19″W / 40.6635°N 73.9387°W / 40.6635; -73.9387 (1 New York City)'}

In [9]:
#converted list into dataframe
df= pd.DataFrame(city_list)

In [10]:
df.head()

,census,change,city,estimate,land_area_km2,land_area_mi,link,location,population_density_km2,population_density_mi,rank,state
0,"8,175,133",+2.74%,New York City[d],"8,398,748",780.9km2,301.5sqmi,/wiki/New_York_City,40°39′49″N 73°56′19″W / 40.6635°N 73.9387°W / ...,"10,933/km2","28,317/sqmi",1,New York
1,"3,792,621",+5.22%,Los Angeles,"3,990,456","1,213.9km2",468.7sqmi,/wiki/Los_Angeles,34°01′10″N 118°24′39″W / 34.0194°N 118.4108°W ...,"3,276/km2","8,484/sqmi",2,California
2,"2,695,598",+0.39%,Chicago,"2,705,994",588.7km2,227.3sqmi,/wiki/Chicago,41°50′15″N 87°40′54″W / 41.8376°N 87.6818°W / ...,"4,600/km2","11,900/sqmi",3,Illinois
3,"2,100,263",+10.72%,Houston[3],"2,325,502","1,651.1km2",637.5sqmi,/wiki/Houston,29°47′12″N 95°23′27″W / 29.7866°N 95.3909°W / ...,"1,395/km2","3,613/sqmi",4,Texas
4,"1,445,632",+14.85%,Phoenix,"1,660,272","1,340.6km2",517.6sqmi,"/wiki/Phoenix,_Arizona",33°34′20″N 112°05′24″W / 33.5722°N 112.0901°W ...,"1,200/km2","3,120/sqmi",5,Arizona


Crawls the links in the given dataframe and stores the page to make scrapping easier as the web page is not repeatedly opened(Space vs time).

Another function to seperate the signature wiki infobox from the given webpage and store it seperately

In [19]:
def crawl_page(x):
    url = baseURL+x
    html = urlopen(url)
    return soup

#locate the info box and extract the data
def get_info_box(x):
    return x.find_all("table",{"class":"infobox"})


Storing all the crawled information in a dataframe and generating csv file.

In [12]:
df["page_info"] = df["link"].apply(crawl_page)
# df.to_csv("page_crawl.csv", sep='\t', encoding='utf-8')

In [13]:
# df = pd.read_csv("page_crawl.csv",sep="\t")

In [14]:
df.head()

,census,change,city,estimate,land_area_km2,land_area_mi,link,location,population_density_km2,population_density_mi,rank,state,page_info
0,"8,175,133",+2.74%,New York City[d],"8,398,748",780.9km2,301.5sqmi,/wiki/New_York_City,40°39′49″N 73°56′19″W / 40.6635°N 73.9387°W / ...,"10,933/km2","28,317/sqmi",1,New York,"<!DOCTYPE html> <html class=""client-nojs"" dir=..."
1,"3,792,621",+5.22%,Los Angeles,"3,990,456","1,213.9km2",468.7sqmi,/wiki/Los_Angeles,34°01′10″N 118°24′39″W / 34.0194°N 118.4108°W ...,"3,276/km2","8,484/sqmi",2,California,"<!DOCTYPE html> <html class=""client-nojs"" dir=..."
2,"2,695,598",+0.39%,Chicago,"2,705,994",588.7km2,227.3sqmi,/wiki/Chicago,41°50′15″N 87°40′54″W / 41.8376°N 87.6818°W / ...,"4,600/km2","11,900/sqmi",3,Illinois,"<!DOCTYPE html> <html class=""client-nojs"" dir=..."
3,"2,100,263",+10.72%,Houston[3],"2,325,502","1,651.1km2",637.5sqmi,/wiki/Houston,29°47′12″N 95°23′27″W / 29.7866°N 95.3909°W / ...,"1,395/km2","3,613/sqmi",4,Texas,"<!DOCTYPE html> <html class=""client-nojs"" dir=..."
4,"1,445,632",+14.85%,Phoenix,"1,660,272","1,340.6km2",517.6sqmi,"/wiki/Phoenix,_Arizona",33°34′20″N 112°05′24″W / 33.5722°N 112.0901°W ...,"1,200/km2","3,120/sqmi",5,Arizona,"<!DOCTYPE html> <html class=""client-nojs"" dir=..."


In [24]:
df["infobox"] = df["page_info"].apply(get_info_box)

1. lambda function(separate_field) - Takes fields from the given list of field_list and fetches the field according to the field list from the info box and stores it in the dataframe
2. lambda function(separate_text_block) -  seperates the text blocks from the webpage according to the field list and stores them in the dataframe according to the field list.

In [43]:
def seperate_field(x,field_list):
    x= x[0]
    tempVariable = x.find_all('tr')
    for i in tempVariable:
        for j in i.find_all("th",limit=10000):
            for k in field_list:
                if k in j.text.lower():
                    return i.find("td").text.replace("\n"," ")

                
def seperate_text_block(x,field_list):
    tempVariable = x.find_all(["h2",'h3',"p"])
    flag = False
    final_string = ""
    for i in tempVariable:
        if (i.name == 'h2' or i.name == 'h3') and field_list in i.text:
            flag = True
            continue
        elif (i.name == 'h2' or i.name == 'h3') and field_list in i.text:
            flag = False
        if flag == True:
            final_string+= i.text + " "
    return final_string
         

Storing all the separated fields in a dataframe

In [25]:
df["county"] = df["infobox"].apply(seperate_field,field_list=["county","counties"])

In [26]:
df["settled"] = df["infobox"].apply(seperate_field,field_list=["settled"])

In [27]:
df["website"] = df["infobox"].apply(seperate_field,field_list=["website"])

In [28]:
df["zip_code"] = df["infobox"].apply(seperate_field,field_list=["zip codes","zip code","zip code(s)"])

In [30]:
df["major_airport"] = df["infobox"].apply(seperate_field,field_list=["major airport(s)","major airport","major airports","primary airport"])

In [31]:
df["demonym"] = df["infobox"].apply(seperate_field,field_list=["demonym(s)"])

In [32]:
df["government_type"] = df["infobox"].apply(seperate_field,field_list=["type"])

In [33]:
df["mayor"] = df["infobox"].apply(seperate_field,field_list=["mayor"])

In [34]:
df["time_zone"] = df["infobox"].apply(seperate_field,field_list=["time zone"])

In [35]:
df["summer_time_zone"] = df["infobox"].apply(seperate_field,field_list=["summer"])

In [36]:
df["area_code"] = df["infobox"].apply(seperate_field,field_list = ["area code"])

In [37]:
df["government_body"] = df["infobox"].apply(seperate_field,field_list= ["body"])

In [44]:
df["history"] = df["page_info"].apply(seperate_text_block,field_list= "History")

In [45]:
df["geography"] = df["page_info"].apply(seperate_text_block,field_list= "Geography")

In [46]:
df["demographics"] = df["page_info"].apply(seperate_text_block,field_list= "Demographics")

In [47]:
df["economy"] = df["page_info"].apply(seperate_text_block,field_list= "Economy")

In [48]:
df["transportation"] = df["page_info"].apply(seperate_text_block,field_list= "Transportation")

In [49]:
df["education"] = df["page_info"].apply(seperate_text_block,field_list = "Education")

In [50]:
df["sports"] = df["page_info"].apply(seperate_text_block,field_list="Sports")

In [51]:
df[["city","sports"]]

,city,sports
0,New York City[d],New York City is home to the headquarters of t...
1,Los Angeles,The city of Los Angeles and its metropolitan a...
2,Chicago,"Sporting News named Chicago the ""Best Sports C..."
3,Houston[3],Houston has sports teams for every major profe...
4,Phoenix,Major league[edit] Phoenix is home to several ...
5,Philadelphia[e],Philadelphia's first professional sports team ...
6,San Antonio,Professional sports[edit] The city's only top-...
7,San Diego,Major League teams Minor League teams College ...
8,Dallas,The Dallas—Fort Worth metropolitan area is hom...
9,San Jose,San Jose is home to the San Jose Sharks of the...


In [52]:
len(df.columns)

33

In [53]:
df.columns

Index(['census', 'change', 'city', 'estimate', 'land_area_km2', 'land_area_mi',
       'link', 'location', 'population_density_km2', 'population_density_mi',
       'rank', 'state', 'page_info', 'infobox', 'county', 'settled', 'website',
       'zip_code', 'major_airport', 'demonym', 'government_type', 'mayor',
       'time_zone', 'summer_time_zone', 'area_code', 'government_body',
       'history', 'geography', 'demographics', 'economy', 'transportation',
       'education', 'sports'],
      dtype='object')

In [57]:
df = df.drop(["infobox","page_info"],axis=1)

In [58]:
df.to_csv("big.csv",index = False)